In [43]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

from urllib.request import urlopen
import json

import os.path


# Lotto 다음 회차 예측 프로그램
- 인공지능 분석 통한 로또 당첨 번호 예측
- 매주 월요일 진행하도록 크롬 작업 진행
    * 첫 시작 / 추가 진행 두 파트로 프로그램 진행되도록 코딩

### 1. API 통해 회차별 당첨번호
- returnValue : 요청결과
- totsellamnt : 총상금액
- drwNoDate : 날짜
- firstWinamnt : 1등 상금액
- firstPrzwnerCo : 1등 당첨자 수
- drwNo : 로또회차
- drwNo1 - 6 : 당첨번호 1 - 6
- bnusNo : 보너스 번호

In [40]:
#url = https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={회차}

response = urlopen("https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=1-1")
response

In [41]:
decode = response.read().decode("utf-8")
data_json = json.loads(decode)
data_json

{'returnValue': 'fail'}

In [37]:
data_json.values()

dict_values([3681782000, 'success', '2002-12-07', 0, 40, 33, 0, 37, 16, 863604600, 1, 23, 29, 10])

In [38]:
data_json.keys()

dict_keys(['totSellamnt', 'returnValue', 'drwNoDate', 'firstWinamnt', 'drwtNo6', 'drwtNo4', 'firstPrzwnerCo', 'drwtNo5', 'bnusNo', 'firstAccumamnt', 'drwNo', 'drwtNo2', 'drwtNo3', 'drwtNo1'])

In [39]:
df = json_normalize(data_json)
df

/var/folders/dx/ny9cjsmd1bb3r42zbp694xkh0000gn/T/ipykernel_1824/2632362630.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(data_json)


,totSellamnt,returnValue,drwNoDate,firstWinamnt,drwtNo6,drwtNo4,firstPrzwnerCo,drwtNo5,bnusNo,firstAccumamnt,drwNo,drwtNo2,drwtNo3,drwtNo1
0,3681782000,success,2002-12-07,0,40,33,0,37,16,863604600,1,23,29,10


In [30]:
# 2회차
response2 = urlopen("https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=2")
decode2 = response2.read().decode("utf-8")
data_json2 = json.loads(decode2)
df2 = json_normalize(data_json2)
df2

/var/folders/dx/ny9cjsmd1bb3r42zbp694xkh0000gn/T/ipykernel_1824/2503244167.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df2 = json_normalize(data_json2)


,totSellamnt,returnValue,drwNoDate,firstWinamnt,drwtNo6,drwtNo4,firstPrzwnerCo,drwtNo5,bnusNo,firstAccumamnt,drwNo,drwtNo2,drwtNo3,drwtNo1
0,4904274000,success,2002-12-14,2002006800,42,25,1,32,2,0,2,13,21,9


In [45]:
dataframe = df.append(df2)
dataframe

,totSellamnt,returnValue,drwNoDate,firstWinamnt,drwtNo6,drwtNo4,firstPrzwnerCo,drwtNo5,bnusNo,firstAccumamnt,drwNo,drwtNo2,drwtNo3,drwtNo1
0,3681782000,success,2002-12-07,0,40,33,0,37,16,863604600,1,23,29,10
0,4904274000,success,2002-12-14,2002006800,42,25,1,32,2,0,2,13,21,9


In [52]:
dataframe.drwNo.iloc[-1]

2

In [ ]:
# 1회차부터 수집해 returnValue가 False일 때까지 수집해 DF으로 만들고 CSV로 저장
def get_data():
    cnt = 1
    while data_json.returnValue == 'fail':
        if cnt == 1:
            response = urlopen(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={cnt}")
            decode = response.read().decode("utf-8")
            data_json = json.loads(decode)
            df = json_normalize(data_json)
        else:
            response = urlopen(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={cnt}")
            decode = response.read().decode("utf-8")
            data_json = json.loads(decode)
            data = json_normalize(data_json)
            df.append(data)
        cnt += 1
    
    return df

# 매주 월요일, 전 주 결과를 추가해 분석하고 당첨번호를 예측해야함
def add_data():
    # 지정된 경로에 csv파일이 있다면?
    csv_path = "data/data.csv"
    if os.path.isfile(csv_path) == True:
            df = pd.read_csv(csv_path)
            last_drwNo = df.drwNo.iloc[-1] 
            while data_json.returnValue == 'fail':
                response = urlopen(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={last_drwNo}")
                decode = response.read().decode("utf-8")
                data_json = json.loads(decode)
                data = json_normalize(data_json)
                df.append(data)
                last_drwNo += 1
    df.to_csv(csv_path)
    return df

